In [44]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
library(themis)
library(parsnip)
options(repr.matrix.max.rows = 6)
source("cleanup.R")

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [45]:
players <- read_csv("https://raw.githubusercontent.com/Snowy129/DSCI100-Project/refs/heads/main/players.csv")
head(players)

sessions <- read_csv("https://raw.githubusercontent.com/Snowy129/DSCI100-Project/refs/heads/main/sessions.csv")
head(sessions)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [46]:
clean_players1 <- players |>
    mutate(gender = as_factor(gender), age = as.integer(age), experience = as_factor(experience)) |>
    select(played_hours, gender, age, experience, subscribe) |>
    filter(age >= 17)
head(clean_players1)




played_hours,gender,age,experience,subscribe
<dbl>,<fct>,<int>,<fct>,<lgl>
3.8,Male,17,Veteran,TRUE
0.0,Male,17,Veteran,FALSE
0.7,Female,21,Amateur,TRUE
0.1,Male,21,Regular,TRUE
0.0,Female,17,Amateur,TRUE
0.0,Female,19,Regular,TRUE


In [48]:
clean_players_with_outliers <- players |>
    mutate(age = as.integer(age)) |>
    select(played_hours,age) |>
    arrange(-played_hours)
head(clean_players_with_outliers)

#Cutting off outliers of played_hours
cutoff_high = quantile(clean_players, probs = 0.975, na.rm = TRUE)
cutoff_high

clean_players <- clean_players_with_outliers |>
    filter(age >= 17) |>
    filter(played_hours < cutoff) |>
    filter(played_hours != 0) |>
    arrange(-played_hours)
clean_players

played_hours,age
<dbl>,<int>
223.1,17
218.1,20
178.2,19
150.0,16
56.1,23
53.9,17


97.5% 
39.95

played_hours,age
<dbl>,<int>
32.0,22
23.7,24
18.5,49
⋮,⋮
0.1,17
0.1,18
0.1,44


In [49]:
set.seed(1234)

#Splitting the Data into Testing and Training
players_split <- initial_split(clean_players, prop = 0.80, strata = played_hours)
players_testing <- testing(players_split)
players_training <- training(players_split)



#PLAN#

- Do first knn regression with all predictors
    - make sure to do vfold and cross analysis
    - find the weights of each predictor and find RMSPE

- Do second knn regression with the top 3 highest weights
    - do vfold
    - find the top 2 weights and the RMSPE

- Do final knn regression with only the top 2 weights
    - do vfold
    - find the RMSPE

- Compare all 3 RMSPE and decide which model is the best


In [50]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("regression")

knn_recipe <- recipe(played_hours ~ age + experience, data = players_training) |>
    step_scale(played_hours, age) |>
    step_center(played_hours, age)

player_vfold <- vfold_cv(players_training, v = 5, strata = played_hours) 

grid_vals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

all_pred_workflow <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = player_vfold, grid = grid_vals) |>
    collect_metrics() |>
    filter(.metric == "rmse")

all_pred_workflow    

ERROR: [1m[33mError[39m in `recipe()`:[22m
[1m[22m[31m✖[39m Misspelled variable name or in-line functions detected.
[36mℹ[39m The following function/misspelling was found: `experience`.
[36mℹ[39m Use steps to do transformations instead.
[36mℹ[39m If your modeling engine uses special terms in formulas, pass that formula to
  workflows as a model formula (`?parsnip::model_formula()`).


In [51]:
knn_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

knn_recipe <- recipe(played_hours ~ age, data = clean_players)

lm_fit <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_spec) |>
    fit(data = clean_players)
lm_fit

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)          age  
      1.198        0.023  


In [54]:
lm_test_results <- lm_fit |>
    predict(clean_players) |>
    bind_cols(clean_players) |>
    metrics(truth = played_hours, estimate = .pred) |>
    filter(.metric == "rmse") |>
    pull(.estimate)

lm_test_results

[1] 4.600081

In [ ]:
# Sample data
set.seed(123)
data <- data.frame(value = rnorm(100))

# Calculate the 97.5th percentile
cutoff <- quantile(data$value, 0.975)

# Filter out the top 2.5%
filtered_data <- data[data$value <= cutoff, ]

# Print the filtered data
print(filtered_data)
